In [4]:
import pandas as pd
import numpy as np

In [5]:
data_path ='../DF/berkeley.csv'
df = pd.read_csv(data_path)

In [6]:
print('Informações gerais do dataset:')
print(df.info())
print('\nPrimeiras linhas:\n', df.head())

Informações gerais do dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12763 entries, 0 to 12762
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Year       12763 non-null  int64 
 1   Major      12763 non-null  object
 2   Gender     12763 non-null  object
 3   Admission  12763 non-null  object
dtypes: int64(1), object(3)
memory usage: 399.0+ KB
None

Primeiras linhas:
    Year  Major Gender Admission
0  1973      C      F  Rejected
1  1973      B      M  Accepted
2  1973  Other      F  Accepted
3  1973  Other      M  Accepted
4  1973  Other      M  Rejected


In [7]:
print('\nVerificação de valores ausentes:')
print(df.isnull().sum())


Verificação de valores ausentes:
Year         0
Major        0
Gender       0
Admission    0
dtype: int64


In [8]:
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] < lower_bound) | (data[column] > upper_bound)]

numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
outliers = {col: detect_outliers_iqr(df, col) for col in numeric_columns}
print('\nOutliers detectados:')
for col, outlier_df in outliers.items():
    print(f'Coluna {col}: {len(outlier_df)} outliers')


Outliers detectados:
Coluna Year: 0 outliers


In [9]:
categorical_columns = df.select_dtypes(include=['object']).columns
for col in categorical_columns:
    print(f'Valores únicos em {col}:', df[col].unique())

Valores únicos em Major: ['C' 'B' 'Other' 'F' 'A' 'D' 'E']
Valores únicos em Gender: ['F' 'M']
Valores únicos em Admission: ['Rejected' 'Accepted']


In [10]:
df['Major'] = df['Major'].astype('category')
df['Gender'] = df['Gender'].astype('category')
df['Admission'] = df['Admission'].astype('category')

In [11]:
# 1. Criar uma variável que categoriza o ano em "década"
df['Decade'] = (df['Year'] // 10) * 10

# 2. Criar uma variável binária para majors "populares" (mais frequentes)
major_counts = df['Major'].value_counts()
threshold = major_counts.mean()  # Threshold baseado na média de frequências
df['Popular_Major'] = df['Major'].apply(lambda x: 1 if major_counts[x] >= threshold else 0)

# 3. Criar uma variável que codifica a admissão como "Aceito" ou "Rejeitado"
df['Admission_Binary'] = df['Admission'].apply(lambda x: 1 if x == 'Accepted' else 0)

# 4. Criar uma variável combinada de gênero e admissão (e.g., "M_Accepted")
df['Gender_Admission'] = df['Gender'].astype(str) + '_' + df['Admission'].astype(str)

In [12]:
print('Informações gerais do dataset:')
print(df.info())
print('\nPrimeiras linhas:\n', df.head())

Informações gerais do dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12763 entries, 0 to 12762
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Year              12763 non-null  int64   
 1   Major             12763 non-null  category
 2   Gender            12763 non-null  category
 3   Admission         12763 non-null  category
 4   Decade            12763 non-null  int64   
 5   Popular_Major     12763 non-null  int64   
 6   Admission_Binary  12763 non-null  category
 7   Gender_Admission  12763 non-null  object  
dtypes: category(4), int64(3), object(1)
memory usage: 449.5+ KB
None

Primeiras linhas:
    Year  Major Gender Admission  Decade  Popular_Major Admission_Binary  \
0  1973      C      F  Rejected    1970              0                0   
1  1973      B      M  Accepted    1970              0                1   
2  1973  Other      F  Accepted    1970              1            

In [14]:
processed_path = '../DF/berkeley_processed.csv'
df.to_csv(processed_path, index=False)
print(f'Dataset pré-processado salvo em {processed_path}')


Dataset pré-processado salvo em ../DF/berkeley_processed.csv
